In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [2]:
#muat dataset colors
colors_df = pd.read_csv('colorhexa_com.csv')
colors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           746 non-null    object 
 1   Hex (24 bit)   746 non-null    object 
 2   Red (8 bit)    746 non-null    int64  
 3   Green (8 bit)  746 non-null    int64  
 4   Blue (8 bit)   746 non-null    int64  
 5   Hue (degrees)  746 non-null    float64
 6   HSL.S (%)      746 non-null    float64
 7   HSL.L (%)      746 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 46.8+ KB


In [3]:
#Dictionary Seasonal
#Spring = Semi
#Summer = Panas
#Autumn = Gugur
#Winter = Dingin
colors_df.head()

,Name,Hex (24 bit),Red (8 bit),Green (8 bit),Blue (8 bit),Hue (degrees),HSL.S (%),HSL.L (%)
0,Air Force blue,#5d8aa8,93,138,168,204.0,30.1,51.2
1,Alice blue,#f0f8ff,240,248,255,208.0,100.0,97.1
2,Alizarin crimson,#e32636,227,38,54,354.9,77.1,52.0
3,Almond,#efdecd,239,222,205,30.0,51.5,87.1
4,Amaranth,#e52b50,229,43,80,348.1,78.2,53.3


In [4]:
#fungsi untuk kategori warna
def categorize_color(row):
    red = row['Red (8 bit)']
    green = row['Green (8 bit)']
    blue = row['Blue (8 bit)']
    hue = row['Hue (degrees)']
    saturation = row['HSL.S (%)']
    lightness = row['HSL.L (%)']
    
    # Spring: Bright and warm colors
    if ((0 <= hue <= 45 or 60 <= hue <= 120) and saturation > 50 and lightness > 50) or (red > 200 and green > 150 and blue < 150):
        return 'Spring'
    # Summer: Cool colors with high lightness
    elif (120 < hue <= 240 and saturation <= 50 and lightness > 50) or (red < 100 and green > 100 and blue > 100):
        return 'Summer'
    # Autumn: Warm colors with low lightness
    elif ((0 <= hue <= 45 or 45 < hue <= 120) and saturation <= 50 and lightness <= 50) or (red > 150 and green < 100 and blue < 100):
        return 'Autumn'
    # Winter: Cool colors with low lightness
    elif (240 < hue <= 360 and saturation > 50 and lightness <= 50) or (red < 100 and green < 100 and blue > 150):
        return 'Winter'
    else:
        return 'Uncategorized'

In [5]:
# Apply the function to categorize colors
colors_df['Season'] = colors_df.apply(categorize_color, axis=1)
colors_df.head(10)

,Name,Hex (24 bit),Red (8 bit),Green (8 bit),Blue (8 bit),Hue (degrees),HSL.S (%),HSL.L (%),Season
0,Air Force blue,#5d8aa8,93,138,168,204.0,30.1,51.2,Summer
1,Alice blue,#f0f8ff,240,248,255,208.0,100.0,97.1,Uncategorized
2,Alizarin crimson,#e32636,227,38,54,354.9,77.1,52.0,Autumn
3,Almond,#efdecd,239,222,205,30.0,51.5,87.1,Spring
4,Amaranth,#e52b50,229,43,80,348.1,78.2,53.3,Autumn
5,Amber,#ffbf00,255,191,0,44.9,100.0,50.0,Spring
6,American rose,#ff033e,255,3,62,346.0,100.0,50.6,Autumn
7,Amethyst,#9966cc,153,102,204,270.0,50.0,60.0,Uncategorized
8,Android Green,#a4c639,164,198,57,74.5,55.3,50.0,Uncategorized
9,Anti-flash white,#f2f3f4,242,243,244,210.0,8.3,95.3,Summer


In [6]:
# Display the categorized data
print(colors_df[['Name', 'Season']])
# Check the distribution of categories
print(colors_df['Season'].value_counts())

                  Name         Season
0       Air Force blue         Summer
1           Alice blue  Uncategorized
2     Alizarin crimson         Autumn
3               Almond         Spring
4             Amaranth         Autumn
..                 ...            ...
741             Yellow         Spring
742      Yellow Orange         Spring
743       Yellow green  Uncategorized
744             Zaffre         Winter
745  Zinnwaldite brown  Uncategorized

[746 rows x 2 columns]
Season
Uncategorized    328
Spring           156
Autumn           110
Summer            91
Winter            61
Name: count, dtype: int64


In [7]:
df = colors_df[colors_df['Season'] != 'Uncategorized']
colors_df.head(10)

,Name,Hex (24 bit),Red (8 bit),Green (8 bit),Blue (8 bit),Hue (degrees),HSL.S (%),HSL.L (%),Season
0,Air Force blue,#5d8aa8,93,138,168,204.0,30.1,51.2,Summer
1,Alice blue,#f0f8ff,240,248,255,208.0,100.0,97.1,Uncategorized
2,Alizarin crimson,#e32636,227,38,54,354.9,77.1,52.0,Autumn
3,Almond,#efdecd,239,222,205,30.0,51.5,87.1,Spring
4,Amaranth,#e52b50,229,43,80,348.1,78.2,53.3,Autumn
5,Amber,#ffbf00,255,191,0,44.9,100.0,50.0,Spring
6,American rose,#ff033e,255,3,62,346.0,100.0,50.6,Autumn
7,Amethyst,#9966cc,153,102,204,270.0,50.0,60.0,Uncategorized
8,Android Green,#a4c639,164,198,57,74.5,55.3,50.0,Uncategorized
9,Anti-flash white,#f2f3f4,242,243,244,210.0,8.3,95.3,Summer
